Effective decorators ~ Decorators and clean code

# 1. Effective Decorators - avoiding common mistakes

## 1.1. Preserving data about the original wrapped object
- problems
  1. 원래 의도한 함수가 아니라 decorator로 감싸진 함수에 대한 property를 갖게 됨
  2. docstring 역시 decorator의 docstring으로 override 됨
    - docstring에 대한 unittest가 돌아가는 경우 문제가 생길 수 있음

In [3]:
import logging
from functools import wraps

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

In [4]:
def trace_decorator(function):
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)

    return wrapped


@trace_decorator
def process_account(account_id):
    """Process an account by Id."""
    logger.info("processing account %s", account_id)
    ...

In [5]:
help(process_account)

Help on function wrapped in module __main__:

wrapped(*args, **kwargs)



In [6]:
process_account.__qualname__

'trace_decorator.<locals>.wrapped'

In [7]:
process_account.__annotations__

{}

* 위 문제를 해결해주기 위한 decorator가 `functools`의 `wraps` 임
* 아래와 같이 변경해주면 앞서 언급한 두 문제가 모두 해결됨
* 저자는 `functools`의 `wraps`를 항상 사용할것을 권장하고 있음

In [8]:
def trace_decorator(function):
    """Log when a function is being called."""

    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)

    return wrapped


@trace_decorator
def process_account(account_id):
    """Process an account by Id."""
    logger.info("processing account %s", account_id)
    ...

In [9]:
help(process_account)

Help on function process_account in module __main__:

process_account(account_id)
    Process an account by Id.



In [10]:
process_account.__qualname__

'process_account'

In [11]:
process_account.__annotations__

{}

## 1.2. Dealing with side effects in decorators
- decorator 함수가 작동하는 과정에서 생길수 있는 문제점들에 관해 설명
  - 의도한 경우가 아니라면 decorator 함수 내에 `innermost` 함수 안에 모든 코드를 작성하는게 좋음
  - decorator가 적용되는 경우는 `run time`이 아니라 `import time`이기 때문에 의도치 않은 부작용이 발생할 수 있음
  - 물론 `import time`에 event를 logging하고 싶은 경우는 위 권장사항을 의도적으로 피하는 경우도 있을 수 있음

In [12]:
import time

In [13]:
def traced_function_wrong(function):
    """An example of a badly defined decorator."""
    logger.debug("started execution of %s", function)
    start_time = time.time()

    @wraps(function)
    def wrapped(*args, **kwargs):
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs", function, time.time() - start_time
        )
        return result

    return wrapped


In [14]:
@traced_function_wrong
def process_with_delay(callback, delay=0):
  time.sleep(delay)
  return callback()

In [15]:
process_with_delay(lambda : None, delay=3)

INFO: function <function process_with_delay at 0x7f8bc9e93560> took 3.02s


In [16]:
process_with_delay(lambda : None, delay=1)

INFO: function <function process_with_delay at 0x7f8bc9e93560> took 4.04s


In [17]:
process_with_delay(lambda : None, delay=3)

INFO: function <function process_with_delay at 0x7f8bc9e93560> took 7.05s


In [18]:
def traced_function(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("started execution of %s", function)
        start_time = time.time()
        result = function(*args, **kwargs)
        logger.info(
            "function %s took %.2fs", function, time.time() - start_time
        )
        return result

    return wrapped

In [19]:
@traced_function
def process_with_delay(callback, delay=0):
  time.sleep(delay)
  return callback()

In [20]:
process_with_delay(lambda : None, delay=3)

INFO: started execution of <function process_with_delay at 0x7f8bc9ea1d40>
INFO: function <function process_with_delay at 0x7f8bc9ea1d40> took 3.00s


In [21]:
process_with_delay(lambda : None, delay=1)

INFO: started execution of <function process_with_delay at 0x7f8bc9ea1d40>
INFO: function <function process_with_delay at 0x7f8bc9ea1d40> took 1.00s


## 1.3. Requiring decorators with side effects
- 1.2 절에서의 side effect가 필요한 경우도 있음
  - 예: event class 들이 정의된 일부 event 만을 사용하고 싶은 경우 (나머지는 중간 단계에 필요한 것들이라고 보면 됨)
    - naive approach: 개별 class들에 대해 flagging을 해서 처리할지 안할지 판단
    - decorator: 필요한 class들을 decoraotor로 감싸서 register에 등록

In [22]:
EVENTS_REGISTRY = {}


def register_event(event_cls):
    """Place the class for the event into the registry to make it accessible in
    the module.
    """
    EVENTS_REGISTRY[event_cls.__name__] = event_cls
    return event_cls


class Event:
    """A base event object"""


class UserEvent:
    TYPE = "user"


@register_event
class UserLoginEvent(UserEvent):
    """Represents the event of a user when it has just accessed the system."""


@register_event
class UserLogoutEvent(UserEvent):
    """Event triggered right after a user abandoned the system."""


def test():
    """
    >>> sorted(EVENTS_REGISTRY.keys()) == sorted(('UserLoginEvent', 'UserLogoutEvent'))
    True
    """

* 예를 들어 `event.py` 에 위 코드를 정의하고 `EVENTS_REGISTRY`를 import 하면
```python
from event import EVENTS_REGISTRY
```
* 아래와 비슷하게 등록된 event들을 가져와서 사용할 수 있음

In [23]:
EVENTS_REGISTRY

{'UserLoginEvent': __main__.UserLoginEvent,
 'UserLogoutEvent': __main__.UserLogoutEvent}

In [24]:
EVENTS_REGISTRY['UserLoginEvent']()

## 1.4. Creating decorators that will always work
- A라는 기능을 하는 함수 B에 사용하기 위한 decorator가 D가 있을때
```python
@D
def B(...):
    ...
```
- C라는 class 내에 A 기능을 하는 method M에 동일하게 D를 사용하면 예상치 못한 에러가 발생할 수 있음
```python
class C():
    @D
    def M(self, ...):
      ...
```
- 아래 예시를 보면,
  - decorator `inject_db_driver`는  
  - string을 입력 받고
  - `DBDriver(string)`을
  - function `run_query`에 대해 입력으로 넘겨 준다

```python
inject_db_driver(run_query())
=> wraps(wrapped())
=> wraps(run_query(DBDriver(dbstring)))
```

In [29]:
class DBDriver:
    def __init__(self, dbstring):
        self.dbstring = dbstring

    def execute(self, query):
        return f"query {query} at {self.dbstring}"


def inject_db_driver(function):
    """This decorator converts the parameter by creating a ``DBDriver``
    instance from the database dsn string.
    """

    @wraps(function)
    def wrapped(dbstring):
        return function(DBDriver(dbstring))

    return wrapped


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function")


class DataHandler:
    """The decorator will not work for methods as it is defined."""

    @inject_db_driver
    def run_query(self, driver):
        return driver.execute(self.__class__.__name__)

In [31]:
run_query('test_OK')

'query test_function at test_OK'

In [33]:
DataHandler().run_query('test_fails')

TypeError: ignored

* 똑같은 기능을하는데 `function`에 대해서는 동작하던 `decorator`가 `method`에 대해 동작하지 않는건 `method`는 self를 첫번째 인자로 받기 때문
* `*args, **kwargs`를 decorator의 signature로 정의해서 이를 해결할수는 있으나 다음의 단점이 존재함
  - 가독성이 떨어짐
  - 주어진 argument에 대한 연산을 하고 싶을때 `*args, **kwargs`는 편리한 방법은 아님
    - argument가 몇 번째 position인지나 key값이 뭔지 알아야 되는 등의 문제점들이 존재

In [64]:
class DBDriver:
    def __init__(self, dbstring):
        self.dbstring = dbstring

    def execute(self, query):
        return f"query {query} at {self.dbstring}"


def inject_db_driver(function):
    """This decorator converts the parameter by creating a ``DBDriver``
    instance from the database dsn string.
    """

    @wraps(function)
    def wrapped(*args, **kwargs):
        print(args)
        print(kwargs)

        return function(*args, driver=DBDriver(kwargs['driver']))

    return wrapped


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function")


class DataHandler:
    """The decorator will not work for methods as it is defined."""

    @inject_db_driver
    def run_query(self, driver):
      
        return driver.execute(self.__class__.__name__)

In [65]:
run_query(driver='test_OK')

()
{'driver': 'test_OK'}


'query test_function at test_OK'

In [66]:
DataHandler().run_query(driver='test_fails')

(<__main__.DataHandler object at 0x7f8bc5cd1b50>,)
{'driver': 'test_fails'}


'query DataHandler at test_fails'

In [118]:
from types import MethodType

class DBDriver:
    def __init__(self, dbstring):
        self.dbstring = dbstring

    def execute(self, query):
        return f"query {query} at {self.dbstring}"


class inject_db_driver:
    """Convert a string to a DBDriver instance and pass this to the wrapped
    function.
    """

    def __init__(self, function):
        # print('function: ', function)
        self.function = function
        wraps(self.function)(self)

    def __call__(self, dbstring):
        # print('dbstring: ', dbstring)
        return self.function(DBDriver(dbstring))

    def __get__(self, instance, owner):
        # print(self.function)
        # print('instance: ', instance)
        # print('owner: ', owner)
        if instance is None:
            return self
        # print('class: ', self.__class__)
        # print('MethodType: ', MethodType(self.function, instance))
        return self.__class__(MethodType(self.function, instance))


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function_2")


class DataHandler:
    @inject_db_driver
    def run_query(self, driver):
        return driver.execute("test_method_2")


In [119]:
run_query('test_OK')

'query test_function_2 at test_OK'

In [120]:
DataHandler().run_query('test_OK')

'query test_method_2 at test_OK'

In [113]:
def my_func(*args, **kwargs):
  print(args)
  print(kwargs)
  return 1

class MyClass:
   pass

In [114]:
obj = MyClass()
obj.func = my_func

In [115]:
obj.func()

()
{}


1

In [116]:
obj2 = MyClass()
obj2.func = MethodType(my_func, obj2)

In [117]:
obj2.func()

(<__main__.MyClass object at 0x7f8bc5bfd390>,)
{}


1